# <h1 align="center"><font color="gree">Building a Data Pipeline in Databricks and using Spark</font></h1>
---

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Link de estudo:

* [Databricks Caderno de anotações](https://docs.databricks.com/aws/pt/notebooks/)

# <font color="red">🚀 Configuração do Databricks Connect</font>

Este notebook está configurado para usar **Databricks Connect**, permitindo executar código localmente mas processando dados no `cluster Databricks`.

## <font color="blue">🔐 Informações necessárias</font>

1. `Server hostname`: URL do seu workspace Databricks
2. `Personal Access Token`: Token de acesso (User Settings → Developer → Access Tokens)
3. `Cluster ID`: ID do cluster ativo (copie da URL quando abrir um cluster)

## <font color="blue">Como configurar?</font>

* `Opção A`: Você pode criar variáveis de ambiente:

```bash
export DATABRICKS_HOST="https://seu-workspace.cloud.databricks.com"
export DATABRICKS_TOKEN="dapi-seu-token-aqui"  
export DATABRICKS_CLUSTER_ID="cluster-id-aqui"
```

* `Opção B`: Crie um arquivo `.env` na pasta do projeto:

```bash
DATABRICKS_HOST=https://seu-workspace.cloud.databricks.com
DATABRICKS_TOKEN=dapi-seu-token-aqui
DATABRICKS_CLUSTER_ID=cluster-id-aqui
```

**⚠️ IMPORTANTE**: Nunca faça commit de tokens/credenciais para Git!

# <font color="red">📋 COMO OBTER AS CREDENCIAIS DO DATABRICKS</font>

## <font color="blue">🔍 1. Server Hostname</font>

- Va para seu workspace Databricks

- Copie a URL do navegador (ex: `https://dbc-a1b2c3d4-e5f6.cloud.databricks.com`)


Você pode copiar a partir do browser só ate `...com`.

## <font color="blue">🔑 2. Personal Access Token</font>

1. No Databricks workspace → clique no seu **avatar** (canto superior direito)

2. **User Settings**

3. **Developer** (no menu lateral)

4. **Access Tokens**

5. **Generate New Token**

6. Dê um nome (ex: "Local Development")

7. Defina expiração (recomendo 90 dias)

8. **Generate** → copie o token (começa com `dapi-...`)

## <font color="blue">💻 3. Cluster ID</font>

1. No Databricks workspace → **Compute** (menu lateral)

2. Clique no cluster que deseja usar

3. Copie o **Cluster ID** da URL ou das configurações do cluster


`NOTA`:

Eu só consegui obter o ID do meu Cluster executando o seguinte comando na própria célula do Databricks:

```python
databricks_cluster_id = spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

print(f"Databricks Cluster ID: {databricks_cluster_id}")
```

# <font color="red">ETL Pipeline com dados armazenados no meu Volume Databricks</font>

In [1]:
import os
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

databricks_host = os.environ['DATABRICKS_HOST']
databricks_token = os.environ['DATABRICKS_TOKEN']
databricks_cluster_id = os.environ['DATABRICKS_CLUSTER_ID']

print("🔗 Carregado minhas credenciais do Databricks, com sucesso!")

🔗 Carregado minhas credenciais do Databricks, com sucesso!


A seguir vamos usar `Databricks Connect` para conectar ao Databricks e executar código `Spark` localmente, mas processando os dados remotamente no cluster Databricks

In [ ]:
from databricks.connect import DatabricksSession # Esta classe é usada para conectar ao Databricks remotamente

spark = DatabricksSession.builder.remote(host=databricks_host, token=databricks_token, cluster_id=databricks_cluster_id).getOrCreate()


print("✅ Conectado ao Databricks com sucesso!")
print(f"🔧 Versão Spark: {spark.version}")

✅ Conectado ao Databricks com sucesso!
🔧 Versão Spark: 4.0.0


A seguir vamos carregar dados do `Volume Databricks`. Basicamente, vou carregar esses dados a partir do `Catalog` do meu workspace Databricks.

In [ ]:
volume_path = "/Volumes/workspace/default_eddy/volumeeddy-tmp-sampledata/sample_data.csv"
print(f"\n📂 Carregando dados do Volume: {volume_path}")

df_spark = spark.read.csv(volume_path, header=True, inferSchema=True)

In [ ]:



print("🎉 DADOS DO VOLUME CARREGADOS COM SUCESSO!")
print("\n📊 Primeiros registros:")
df_spark.show()

print(f"\n📈 Informações do Dataset:")
print(f"   • Linhas: {df_spark.count()}")
print(f"   • Colunas: {len(df_spark.columns)}")
print(f"   • Nomes das Colunas: {df_spark.columns}")

print("\n📋 Schema do Dataset:")
df_spark.printSchema()

# Converter para Pandas para análises adicionais (opcional)
df_pandas = df_spark.toPandas()
print(f"\n✅ Dados também disponíveis como Pandas DataFrame para análises locais!")


In [ ]:
# 📊 ANÁLISE AVANÇADA DOS DADOS DO VOLUME DATABRICKS
print("="*70)
print("📊 ANÁLISE COMPLETA DOS DADOS DO VOLUME DATABRICKS")
print("="*70)

# Estatísticas descritivas com Spark
print("\n📊 Estatísticas Descritivas (processadas no cluster Databricks):")
df_spark.describe().show()

# Análises com Spark SQL
print("\n🔍 Análises com Spark SQL:")

# Registrar como view temporária para usar SQL
df_spark.createOrReplaceTempView("pessoas")

# 1. Análise de idade
print("\n👥 Análise de Idade:")
idade_stats = spark.sql("""
    SELECT 
        AVG(Age) as idade_media,
        MIN(Age) as idade_minima,
        MAX(Age) as idade_maxima,
        COUNT(*) as total_pessoas
    FROM pessoas
""")
idade_stats.show()

# 2. Maiores de idade
print("\n🔞 Pessoas maiores de idade:")
adults = spark.sql("SELECT * FROM pessoas WHERE Age >= 18")
adults.show()

adult_count = adults.count()
total_count = df_spark.count()
print(f"📈 {adult_count} de {total_count} pessoas são maiores de idade ({adult_count/total_count*100:.1f}%)")

# 3. Análise por cidade
print("\n🏙️ Contagem por Cidade:")
city_analysis = spark.sql("""
    SELECT City, COUNT(*) as quantidade
    FROM pessoas 
    GROUP BY City 
    ORDER BY quantidade DESC
""")
city_analysis.show()

# 4. Análises avançadas com Window Functions
print("\n📈 Análises Avançadas:")
advanced_analysis = spark.sql("""
    SELECT 
        Name,
        Age,
        City,
        CASE 
            WHEN Age >= 18 THEN 'Adulto'
            ELSE 'Menor de idade'
        END as categoria_idade,
        ROW_NUMBER() OVER (PARTITION BY City ORDER BY Age DESC) as ranking_idade_cidade
    FROM pessoas
    ORDER BY City, Age DESC
""")
advanced_analysis.show()

# Converter para Pandas para análises complementares
print("\n🐼 Análises complementares com Pandas:")
df_pandas = df_spark.toPandas()
print(f"   • Dataset convertido: {df_pandas.shape[0]} linhas, {df_pandas.shape[1]} colunas")
print(f"   • Idade média: {df_pandas['Age'].mean():.2f} anos")
print(f"   • Cidades únicas: {df_pandas['City'].nunique()}")

print("\n✅ Pipeline ETL com Databricks Volume concluído com sucesso!")
print("   • 🌩️ Processamento: Cluster Databricks")
print("   • 📁 Fonte: Volume Databricks")
print(f"   • 📊 Registros: {df_spark.count()}")
print(f"   • 🚀 Engine: Spark {spark.version}")
print("   • 💾 Disponível em: Spark DataFrame + Pandas DataFrame")
